In [95]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from torch.autograd import Variable
import sys
import random
from torch import nn
from torchvision import models
from torchvision.models import ResNeXt50_32X4D_Weights
import glob

In [96]:
path_to_videos = ['C:/Users/USER/Downloads/Authentica---DeepFake-detection/testcases/WhatsApp Video 2025-04-11 at 13.29.25_7e6ebfc1.mp4']

#path_to_videos = ['D:/btech/research/app_copy/app_copy/video/full fake.mp4']

frame_count = []
for video_file in path_to_videos:
  cap = cv2.VideoCapture(video_file)
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames are " , frame_count)

frames are  [375]


In [97]:
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(x_lstm[:,-1,:]))

In [98]:
im_size = 112
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
sm = nn.Softmax(dim=1)
inv_normalize =  transforms.Normalize(mean=-1*np.divide(mean,std),std=np.divide([1,1,1],std))
def im_convert(tensor):
    """ Display a tensor as an image. """
    image = tensor.to("cpu").clone().detach()
    image = image.squeeze()
    image = inv_normalize(image)
    image = image.numpy()
    image = image.transpose(1,2,0)
    image = image.clip(0, 1)
    cv2.imwrite('./2.png',image*255)
    return image

In [99]:

# Initialize lists to store predictions and confidence values for each frame


In [100]:
def predict(model, img, path='./'):
    # Ensure the model is in evaluation mode and the input is on the correct device
    model.eval()

    if not isinstance(img, torch.Tensor) or img.shape[0] == 0:
        print("Warning: Empty or invalid input image tensor.")
        return None

    # Ensure the input is on the correct device
    img = img.to('cuda')

    # Loop through each frame in the batch
    for i in range(img.size(1)):  # img.size(1) gives the number of frames
        frame = img[:, i, :, :, :].unsqueeze(0)  # Extract each frame and add batch dimension

        # Forward pass
        fmap, logits = model(frame)

        # Extract weights from the final linear layer
        try:
            weight_softmax = model.linear1.weight.detach().cpu().numpy()
        except AttributeError:
            print("Error: Model does not have a layer named 'linear1'.")
            return None

        # Softmax and prediction
        logits = torch.nn.functional.softmax(logits, dim=1)  # Ensure softmax is applied
        _, prediction = torch.max(logits, 1)
        confidence = logits[:, int(prediction.item())].item() * 100

        # Store results
        if int(prediction.item()) == 1:
            predictions.append(1)
        else:
            predictions.append(0)
        confidences.append(confidence)

        #print(f'Frame {i + 1}: Prediction = {prediction.item()}, Confidence = {confidence:.2f}%')

    # Optionally, return the list of predictions and confidences
    return [predictions, confidences]



In [101]:
#new implementation

class validation_dataset(Dataset):
    def __init__(self,video_names,sequence_length = 60,transform = None):
        self.video_names = video_names
        self.transform = transform
        self.count = sequence_length
    def __len__(self):
        return len(self.video_names)
    def __getitem__(self, idx):
        video_path = self.video_names[idx]
        frames = []

        # Extract all frames from the video
        for frame in self.frame_extract(video_path):
            frames.append(self.transform(frame))
        #print("frames: ",frames,"\n Length of frames: ",len(frames))
        # Stack all frames into a tensor
        frames = torch.stack(frames)
        #print("frames: ",frames,"\n Length of frames: ",len(frames))
        # Add an extra dimension (batch dimension)
        #print("frames: ",frames,"\n Length of frames: ",len(frames))
        return frames.unsqueeze(0)


    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path) 
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image
              
def im_plot(tensor):
    image = tensor.cpu().numpy().transpose(1, 2, 0)
    b, g, r = cv2.split(image)
    image = cv2.merge((r, g, b))
    image = image * [0.22803, 0.22145, 0.216989] + [0.43216, 0.394666, 0.37645]
    image = image * 255.0
    plt.imshow(image.astype(int))
    plt.show()

In [102]:
train_transforms = transforms.Compose([
  transforms.ToPILImage(),
  transforms.Resize((im_size,im_size)),
  transforms.ToTensor(),
  transforms.Normalize(mean,std)])

In [103]:

video_dataset = validation_dataset(path_to_videos,sequence_length = 20,transform = train_transforms)


#print("hi")

In [104]:
'''print(len(predictions))
print(len(confidences))
for i in range(0,len(predictions)):
    print(predictions[i], confidences[i] )'''

'print(len(predictions))\nprint(len(confidences))\nfor i in range(0,len(predictions)):\n    print(predictions[i], confidences[i] )'

In [105]:
import os

# Path to your folder
folder_path = 'C:/Users/USER/Downloads/Authentica---DeepFake-detection/models'

# List to store filenames
model_names = []

# Loop through items in folder
for item in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, item)):
        model_names.append(item)

print(model_names)
len(model_names)


['train_model_71.26_epoch_1.pt', 'train_model_76.19_epoch_2.pt', 'train_model_79.03_epoch_3.pt', 'train_model_81.47_epoch_4.pt', 'train_model_84.24_epoch_5.pt', 'train_model_85.42_epoch_6.pt', 'train_model_87.32_epoch_7.pt', 'train_model_88.43_epoch_8.pt', 'train_model_90.19_epoch_9.pt', 'train_model_91.52_epoch_10.pt', 'train_model_92.00_epoch_11.pt', 'train_model_92.95_epoch_12.pt', 'train_model_93.47_epoch_13.pt', 'train_model_93.68_epoch_14.pt', 'train_model_94.83_epoch_15.pt', 'train_model_94.93_epoch_16.pt', 'train_model_95.32_epoch_17.pt', 'train_model_95.74_epoch_19.pt', 'train_model_95.78_epoch_18.pt', 'train_model_96.12_epoch_20.pt', 'train_model_96.48_epoch_21.pt', 'train_model_96.64_epoch_23.pt', 'train_model_96.88_epoch_22.pt', 'train_model_97.17_epoch_24.pt', 'train_model_97.36_epoch_25.pt', 'train_model_97.37_epoch_28.pt', 'train_model_97.40_epoch_26.pt', 'train_model_97.44_epoch_27.pt', 'train_model_97.63_epoch_29.pt', 'train_model_97.65_epoch_43.pt', 'train_model_97.84

102

In [106]:
for z in model_names:
    predictions = []
    confidences = []
    path_to_model = 'C:/Users/USER/Downloads/Authentica---DeepFake-detection/models/'+z
    model = Model(2).cuda()
    model.load_state_dict(torch.load(path_to_model))
    # Ensure the model is in evaluation mode
    model.eval()
    # Initialize sum to accumulate confidence values
    sum_confidence = 0

    # Process the single video
    p = predict(model, video_dataset[0], './')  # Assuming there's only one video in video_dataset

    if p is not None:
        predictions = p[0]
        confidences = p[1]

        # Accumulate confidence values based on predictions
        for i in range(len(predictions)):
            if predictions[i] == 1:  # Assuming 1 means 'real'
                sum_confidence += confidences[i]
            else:  # Assuming 0 means 'fake'
                sum_confidence += 100 - confidences[i]

        # Calculate the average confidence
        avg_confidence = sum_confidence / len(predictions)

        # Determine the final prediction for the video
        if avg_confidence < 50 :
            fake_confidence = 100 - avg_confidence
            #if fake_confidence > 95:
            print(f"\033[31mModel: {z} | Fake | Confidence: {fake_confidence:.2f}%\033[0m")
        else:
            #if avg_confidence > 95:
            print(f"\033[32mModel: {z} | Real | Confidence: {avg_confidence:.2f}%\033[0m")
    else:
        print("Prediction failed for the video.")



Model: train_model_71.26_epoch_1.pt | Fake | Confidence: 70.19%
Model: train_model_76.19_epoch_2.pt | Real | Confidence: 64.95%
Model: train_model_79.03_epoch_3.pt | Real | Confidence: 59.41%
Model: train_model_81.47_epoch_4.pt | Real | Confidence: 57.72%
Model: train_model_84.24_epoch_5.pt | Fake | Confidence: 56.80%
Model: train_model_85.42_epoch_6.pt | Fake | Confidence: 65.40%
Model: train_model_87.32_epoch_7.pt | Fake | Confidence: 54.58%
Model: train_model_88.43_epoch_8.pt | Fake | Confidence: 64.83%
Model: train_model_90.19_epoch_9.pt | Real | Confidence: 52.69%
Model: train_model_91.52_epoch_10.pt | Real | Confidence: 51.16%
Model: train_model_92.00_epoch_11.pt | Fake | Confidence: 58.91%
Model: train_model_92.95_epoch_12.pt | Fake | Confidence: 54.93%
Model: train_model_93.47_epoch_13.pt | Fake | Confidence: 53.56%
Model: train_model_93.68_epoch_14.pt | Fake | Confidence: 65.36%
Model: train_model_94.83_epoch_15.pt | Real | Confidence: 54.86%
Model: train_model_94.93_epoch_16.

Model: train_model_71.26_epoch_1.pt | Fake | Confidence: 53.13%
Model: train_model_76.19_epoch_2.pt | Fake | Confidence: 62.07%
Model: train_model_79.03_epoch_3.pt | Fake | Confidence: 68.66%
Model: train_model_81.47_epoch_4.pt | Fake | Confidence: 78.57%
Model: train_model_84.24_epoch_5.pt | Fake | Confidence: 87.49%
Model: train_model_85.42_epoch_6.pt | Fake | Confidence: 72.83%
Model: train_model_87.32_epoch_7.pt | Fake | Confidence: 83.60%
Model: train_model_88.43_epoch_8.pt | Fake | Confidence: 87.58%
Model: train_model_90.19_epoch_9.pt | Fake | Confidence: 76.70%
Model: train_model_91.52_epoch_10.pt | Fake | Confidence: 77.86%
Model: train_model_92.00_epoch_11.pt | Fake | Confidence: 85.41%
Model: train_model_92.95_epoch_12.pt | Fake | Confidence: 86.32%
Model: train_model_93.47_epoch_13.pt | Fake | Confidence: 83.02%
Model: train_model_93.68_epoch_14.pt | Fake | Confidence: 89.67%
Model: train_model_94.83_epoch_15.pt | Fake | Confidence: 73.44%
Model: train_model_94.93_epoch_16.pt | Fake | Confidence: 90.21%
Model: train_model_95.32_epoch_17.pt | Fake | Confidence: 90.42%
Model: train_model_95.74_epoch_19.pt | Fake | Confidence: 92.21%
Model: train_model_95.78_epoch_18.pt | Fake | Confidence: 89.82%
Model: train_model_96.12_epoch_20.pt | Fake | Confidence: 93.86%
Model: train_model_96.48_epoch_21.pt | Fake | Confidence: 91.51%
Model: train_model_96.64_epoch_23.pt | Fake | Confidence: 77.44%
Model: train_model_96.88_epoch_22.pt | Fake | Confidence: 95.64%
Model: train_model_97.17_epoch_24.pt | Fake | Confidence: 90.44%
Model: train_model_97.36_epoch_25.pt | Fake | Confidence: 95.71%
Model: train_model_97.37_epoch_28.pt | Fake | Confidence: 95.39%
Model: train_model_97.40_epoch_26.pt | Fake | Confidence: 92.83%
Model: train_model_97.44_epoch_27.pt | Fake | Confidence: 95.36%
Model: train_model_97.63_epoch_29.pt | Fake | Confidence: 91.32%
Model: train_model_97.65_epoch_43.pt | Fake | Confidence: 80.70%
Model: train_model_97.84_epoch_31.pt | Fake | Confidence: 94.93%
Model: train_model_97.88_epoch_32.pt | Fake | Confidence: 86.35%
Model: train_model_98.04_epoch_35.pt | Fake | Confidence: 89.12%
Model: train_model_98.09_epoch_33.pt | Fake | Confidence: 85.34%
Model: train_model_98.17_epoch_30.pt | Fake | Confidence: 96.26%
Model: train_model_98.19_epoch_44.pt | Fake | Confidence: 80.15%
Model: train_model_98.20_epoch_39.pt | Fake | Confidence: 85.95%
Model: train_model_98.33_epoch_37.pt | Fake | Confidence: 89.73%
Model: train_model_98.35_epoch_38.pt | Fake | Confidence: 92.34%
Model: train_model_98.36_epoch_34.pt | Fake | Confidence: 89.97%
Model: train_model_98.43_epoch_45.pt | Fake | Confidence: 86.34%
Model: train_model_98.53_epoch_41.pt | Fake | Confidence: 75.68%
Model: train_model_98.54_epoch_36.pt | Fake | Confidence: 85.30%
Model: train_model_98.54_epoch_46.pt | Fake | Confidence: 88.55%
Model: train_model_98.55_epoch_51.pt | Fake | Confidence: 92.93%
Model: train_model_98.66_epoch_40.pt | Fake | Confidence: 90.11%
Model: train_model_98.66_epoch_47.pt | Fake | Confidence: 91.01%
Model: train_model_98.69_epoch_49.pt | Fake | Confidence: 88.96%
Model: train_model_98.71_epoch_48.pt | Fake | Confidence: 92.88%
Model: train_model_98.74_epoch_42.pt | Fake | Confidence: 88.82%
Model: train_model_98.79_epoch_50.pt | Fake | Confidence: 93.46%
Model: train_model_98.82_epoch_52.pt | Fake | Confidence: 93.11%
Model: train_model_98.91_epoch_56.pt | Fake | Confidence: 90.28%
Model: train_model_98.97_epoch_57.pt | Fake | Confidence: 90.03%
Model: train_model_99.00_epoch_74.pt | Fake | Confidence: 87.37%
Model: train_model_99.02_epoch_53.pt | Fake | Confidence: 93.76%
Model: train_model_99.02_epoch_63.pt | Fake | Confidence: 82.42%
Model: train_model_99.04_epoch_65.pt | Fake | Confidence: 81.96%
Model: train_model_99.05_epoch_58.pt | Fake | Confidence: 86.74%
Model: train_model_99.07_epoch_61.pt | Fake | Confidence: 84.99%
Model: train_model_99.07_epoch_67.pt | Fake | Confidence: 88.46%
Model: train_model_99.10_epoch_55.pt | Fake | Confidence: 88.82%
Model: train_model_99.11_epoch_66.pt | Fake | Confidence: 86.10%
Model: train_model_99.12_epoch_54.pt | Fake | Confidence: 89.15%
Model: train_model_99.12_epoch_62.pt | Fake | Confidence: 91.57%
Model: train_model_99.12_epoch_72.pt | Fake | Confidence: 95.89%
Model: train_model_99.13_epoch_70.pt | Fake | Confidence: 90.24%
Model: train_model_99.19_epoch_59.pt | Fake | Confidence: 90.53%
Model: train_model_99.20_epoch_64.pt | Fake | Confidence: 88.06%
Model: train_model_99.24_epoch_73.pt | Fake | Confidence: 91.96%
Model: train_model_99.24_epoch_80.pt | Fake | Confidence: 91.51%
Model: train_model_99.24_epoch_83.pt | Fake | Confidence: 95.05%
Model: train_model_99.25_epoch_77.pt | Fake | Confidence: 86.94%
Model: train_model_99.27_epoch_75.pt | Fake | Confidence: 96.68%
Model: train_model_99.27_epoch_81.pt | Fake | Confidence: 94.75%
Model: train_model_99.29_epoch_68.pt | Fake | Confidence: 95.60%
Model: train_model_99.29_epoch_90.pt | Fake | Confidence: 90.08%
Model: train_model_99.31_epoch_71.pt | Fake | Confidence: 85.22%
Model: train_model_99.34_epoch_60.pt | Fake | Confidence: 91.94%
Model: train_model_99.34_epoch_78.pt | Fake | Confidence: 92.28%
Model: train_model_99.34_epoch_79.pt | Fake | Confidence: 90.67%
Model: train_model_99.34_epoch_82.pt | Fake | Confidence: 91.39%
Model: train_model_99.35_epoch_92.pt | Fake | Confidence: 96.06%
Model: train_model_99.36_epoch_76.pt | Fake | Confidence: 95.01%
Model: train_model_99.37_epoch_69.pt | Fake | Confidence: 92.58%
Model: train_model_99.41_epoch_100.pt | Fake | Confidence: 94.84%
Model: train_model_99.41_epoch_94.pt | Fake | Confidence: 96.30%
Model: train_model_99.42_epoch_91.pt | Fake | Confidence: 96.72%
Model: train_model_99.43_epoch_85.pt | Fake | Confidence: 95.23%
Model: train_model_99.45_epoch_87.pt | Fake | Confidence: 96.00%
Model: train_model_99.49_epoch_88.pt | Fake | Confidence: 95.76%
Model: train_model_99.49_epoch_97.pt | Fake | Confidence: 90.45%
Model: train_model_99.50_epoch_86.pt | Fake | Confidence: 96.16%
Model: train_model_99.50_epoch_96.pt | Fake | Confidence: 95.84%
Model: train_model_99.51_epoch_89.pt | Fake | Confidence: 98.15%
Model: train_model_99.52_epoch_98.pt | Fake | Confidence: 88.16%
Model: train_model_99.52_epoch_99.pt | Fake | Confidence: 97.75%
Model: train_model_99.59_epoch_101.pt | Fake | Confidence: 96.71%
Model: train_model_99.59_epoch_95.pt | Fake | Confidence: 96.12%
Model: train_model_99.65_epoch_102.pt | Fake | Confidence: 96.82%
Model: train_model_99.67_epoch_84.pt | Fake | Confidence: 95.44%
Model: train_model_99.69_epoch_93.pt | Fake | Confidence: 96.59%

In [107]:
'''# Convert to TorchScript
scripted_model = torch.jit.script(model)
scripted_model.save("C:/Users/alapa/Documents/Deepfake_detection_using_deep_learning-master/Deepfake_detection_using_deep_learning-master/model_scripted.pt")
'''

'# Convert to TorchScript\nscripted_model = torch.jit.script(model)\nscripted_model.save("C:/Users/alapa/Documents/Deepfake_detection_using_deep_learning-master/Deepfake_detection_using_deep_learning-master/model_scripted.pt")\n'